In [7]:
import pickle
import numpy as np
from numpy import array, ones,zeros
import sys


In [8]:
def deletion_viterbi2(c_tags,proba_inits,proba_tran,proba_obs,obs):
    L = len(obs)
    V = dict()
    psi = dict()
    # if p(y0)*p(x0|y0)*p(y1|y0)*p(x1|y1)<p(additional_y0)*p(y0|additional_y0)*p(x0|y0)*p(y1|y0)*p(x1|y1) ,we add a state
    for y in c_tags:
        
        V[0,y] = proba_inits[y] * proba_obs[(obs[0],y)]
        psi[0,y] = 'Null'
    mark = dict()
    for t in range(L):
        for j in c_tags:
            mark[(t,j)] = 0
    for t in range(1,L-1):
        for y in c_tags:
            if mark[(t,y)]==0:
            
                (prob1, state1) = max([(V[t-1,y0] * proba_tran[y0,y] * proba_obs[obs[t],y], y0) for y0 in c_tags]) 
                (prob2, state2) = max([(V[t-1,y1] * proba_tran[y1,y] * proba_obs[obs[t+1],y],y1) for y1 in c_tags])
                if prob2 > prob1:
                    V[t,y] = prob2
                    psi[t,y] = state1
                    mark[t+1,y] = 1
                    V[t+1,y] = prob2
                    psi[t+1,y] = (state2,'jump')
                else:
                    psi[t,y] = state1
                    V[t,y] = prob1
    for y in c_tags:
        if mark[(L-1,y)]==0:
            (prob1, state1) = max([(V[L-2,y0] * proba_tran[y0,y] * proba_obs[obs[L-1],y], y0) for y0 in c_tags])
            psi[L-1,y] = state1
            V[L-1,y] = prob1
    tmp = []
    result = []
    for i in c_tags:
        tmp.append((V[L-1,i],i))
    last = max(tmp)
    t = L-1
    result.append(last[1]) 
    last = psi[t,result[-1]]
    while t>=1:
        if len(last)==2:
            result.append(last[0])
            last = psi[t-1,result[-1]]
            t=t-2
        else:
            result.append(last)
            last = psi[t,result[-1]]
            t = t -1
    result.reverse()
    return result

In [9]:
with open('train10.pkl','rb') as f:
    data = pickle.load(f)
def make_counts(corpus):
    c_tags = dict()
    c_obs = dict()
    c_trans = dict()
    c_pairs = dict()
    c_inits = dict()
    for word in corpus:
        for i in range(len(word)):
            pair = word[i]
            tag = pair[1]
            obs = pair[0]
            if tag in c_tags:
                c_tags[tag]+=1
            else:
                c_tags[tag]=1
            if obs in c_obs:
                c_obs[obs]+=1
            else:
                c_obs[obs]=1
            if pair in c_pairs:
                c_pairs[pair]+=1
            else:
                c_pairs[pair]=1
            if i>0:
                trans = (word[i-1][1],tag)
                if trans in c_trans:
                    c_trans[trans]+=1
                else:
                    c_trans[trans]=1
            else:
                if tag in c_inits:
                    c_inits[tag]+=1
                else:
                    c_inits[tag]=1
    return c_obs, c_tags, c_pairs, c_trans, c_inits

In [10]:
c_obs, c_tags, c_pairs, c_trans, c_inits = make_counts(data)
s = 0
proba_inits = dict.fromkeys(c_tags)
for i in proba_inits:
    proba_inits[i] = 0
for i in c_inits:
    s = s + c_inits[i]
for i in c_inits:
    proba_inits[i] = c_inits[i]/s
from __future__ import division
proba_tran = dict()
s = dict()
for i in c_tags:
    s[(i)] = 0
    for j in c_tags:
        proba_tran[(i,j)] = 0
for i in c_trans:
    s[i[0]] = s [i[0]] + c_trans[i]
for i in c_trans:
    proba_tran[i] = c_trans[i]/s[i[0]]
proba_obs = dict()
for i in c_tags:
    for j in c_tags:
        proba_obs[(i,j)]=0
for i in c_pairs:
    proba_obs[i] = c_pairs[i]/c_tags[i[1]]

In [12]:
obs1 = ['t','w','h','e']
state = ['t','h','e']

print "the observation is  ", obs1
print "the actual state is   ", state
print "our return is ", deletion_viterbi2(c_tags,proba_inits,proba_tran,proba_obs,obs1)

print "==================================================================================="

obs2 = ['z','e','x','d']
print "the observation is  ", obs2
print "our return is ", deletion_viterbi2(c_tags,proba_inits,proba_tran,proba_obs,obs2)
print "==================================================================================="

obs3 = ['m','a','r','c','k']
print "the observation is  ",obs3
print "our return is ",deletion_viterbi2(c_tags,proba_inits,proba_tran,proba_obs,obs3)


the observation is   ['t', 'w', 'h', 'e']
the actual state is    ['t', 'h', 'e']
our return is  ['t', 'h', 'e']
the observation is   ['z', 'e', 'x', 'd']
our return is  ['z', 'e', 'd']
the observation is   ['m', 'a', 'r', 'c', 'k']
our return is  ['a', 'c', 'k']
